In [ ]:
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core.conda_dependencies import CondaDependencies 
from azureml.core.webservice import AciWebservice
import os

In [ ]:
### connecting to ML workspace
### --------------------

ws = Workspace.from_config()

In [ ]:
### downloading model
### --------------------

os.remove('model.pkl')

model = Model(ws, 'german-credit-hsg')
model.download(target_dir=os.getcwd(), exist_ok=True)

# verify the downloaded model file
file_path = os.path.join(os.getcwd(), "model.pkl")


In [ ]:
### adding dependencies
### --------------------

myenv = CondaDependencies()
myenv.add_conda_package("scikit-learn")
#myenv.add_pip_package("joblib")
#myenv.add_pip_package("pandas")

with open("myenv.yml", "w") as f:
    f.write(myenv.serialize_to_string())

with open("myenv.yml", "r") as f:
    print(f.read())

In [ ]:
### web service config
### --------------------

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "german-credit-hsg",  
                                                     "framework": "sklearn"},
                                               description='Predict credit risk with sklearn', 
                                               enable_app_insights = True,
                                               location = 'westeurope')

In [ ]:
%%writefile scripts/score.py

### scoring script
### --------------------

import json
import numpy as np
import pandas as pd
import joblib
from azureml.core.model import Model

MODEL_NAME = 'german-credit-hsg'

def init():
    global model
    model_path = Model.get_model_path(MODEL_NAME)
    model = joblib.load(model_path)

def run(raw_data):
    try:
        data = json.loads(raw_data)['data']
        input_df = pd.DataFrame.from_dict(data)
        proba = model.predict_proba(input_df)
        result = {"predict_proba":proba.tolist()}
        return result
    except Exception as e:
        error = str(e)
        return error

In [ ]:

%%time
from azureml.core.webservice import Webservice
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(runtime= "python", 
                                   entry_script="./scripts/score.py", 
                                   conda_file="myenv.yml")

service = Model.deploy(workspace=ws, 
                       name='german-credit-hsg',
                       models=[model], 
                       inference_config=inference_config,
                       deployment_config=aciconfig)

service.wait_for_deployment(show_output=True)